In [1]:
import pandas as pd
import numpy as np

In [2]:
print(pd.__version__)
print(np.__version__)

1.4.3
1.21.2


##### points to notify


- trade-off in missing data conventions
- none pythonic missing data
- NaN missing numerical data
- NaN and None in pandas

##### Handling Missing Data

The difference between data found in many tutorials and data in the real world is that
real-world data is rarely clean and homogeneous. In particular, many interesting
datasets will have some amount of data missing. To make matters even more compli‐
cated, different data sources may indicate missing data in different ways.

##### Trade-Offs in Missing Data Conventions

A number of schemes have been developed to indicate the presence of missing data in
a table or DataFrame. Generally, they revolve around one of two strategies: using a
mask that globally indicates missing values, or choosing a sentinel value that indicates
a missing entry. <br>

In the masking approach, the mask might be an entirely separate Boolean array, or it
may involve appropriation of one bit in the data representation to locally indicate the
null status of a value. <br>

In the sentinel approach, the sentinel value could be some data-specific convention,
such as indicating a missing integer value with –9999 or some rare bit pattern, or it
could be a more global convention, such as indicating a missing floating-point value
with NaN (Not a Number), a special value which is part of the IEEE floating-poinspecification. <br>

None of these approaches is without trade-offs: use of a separate mask array requires
allocation of an additional Boolean array, which adds overhead in both storage and
computation. A sentinel value reduces the range of valid values that can be repre‐
sented, and may require extra (often non-optimized) logic in CPU and GPU arith‐
metic. Common special values like NaN are not available for all data types. <br>

As in most cases where no universally optimal choice exists, different languages and
systems use different conventions. For example, the R language uses reserved bit pat‐
terns within each data type as sentinel values indicating missing data, while the SciDB
system uses an extra byte attached to every cell to indicate a NA state.

##### Missing Data in Pandas

The way in which Pandas handles missing values is constrained by its reliance on the
NumPy package, which does not have a built-in notion of NA values for nonfloating-point
data types ikely even requiring a new fork of the NumPy package. Further, for the smaller data
types (such as 8-bit integers), sacrificing a bit to use as a mask will significantly
reduce the range of values it can represent. <br>

NumPy does have support for masked arrays—that is, arrays that have a separate
Boolean mask array attached for marking data as “good” or “bad.” Pandas could have
derived from this, but the overhead in both storage, computation, and code mainte‐
nance makes that an unattractive choice. <br>

With these constraints in mind, Pandas chose to use sentinels for missing data, and
further chose to use two already-existing Python null values: the special floatingpoint
NaN value, and the Python
None object. This choice has some side effects, as
we
will see, but in practice ends up being a good
compromise in most cases of
interest.

##### None: Pythonic missing data

The first sentinel value used by Pandas is None, a Python singleton object that is often
used for missing data in Python code. Because None is a Python object, it cannot be
used in any arbitrary NumPy/Pandas array, but only in arrays with data type
'object' (i.e., arrays of Python objects):

In [3]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

This dtype=object means that the best common type representation NumPy could
infer for the contents of the array is that they are Python objects. While this kind of
object array is useful for some purposes, any operations on the data will be done at
the Python level, with much more overhead than the typically fast operations seen for
arrays with native types:

In [4]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
69.4 ms ± 1.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.17 ms ± 15.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



The use of Python objects in an array also means that if you perform aggregations
like sum() or min() across an array with a None value, you will generally get an error:

In [5]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

This reflects the fact that addition between an integer and None is undefined.

##### NaN: Missing numerical data

The other missing data representation, NaN (acronym for Not a Number), is differenit is a special floating-point value recognized by all systems that use the standard
IEEE floating-point representation:

In [6]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

Notice that NumPy chose a native floating-point type for this array: this means that
unlike the object array from before, this array supports fast operations pushed into
compiled code. You should be aware that NaN is a bit like a data virus—it infects any
other object it touches. Regardless of the operation, the result of arithmetic with NaN
will be another NaN:

In [7]:
1 + np.NaN

nan

In [8]:
0 * np.NaN

nan

Note that this means that aggregates over the values are well defined (i.e., they don’t
result in an error) but not always useful:

In [9]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy does provide some special aggregations that will ignore these missing values:

In [10]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

Keep in mind that NaN is specifically a floating-point value; there is no equivalent
NaN value for integers, strings, or other types.

#####  NaN and None in Pandas

NaN and None both have their place, and Pandas is built to handle the two of them
nearly interchangeably, converting between them where appropriate:

In [11]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types that don’t have an available sentinel value, Pandas automatically type-casts
when NA values are present. For example, if we set a value in an integer array to
np.nan, it will automatically be upcast to a floating-point type to accommodate the
NA:

In [12]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [13]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Notice that in addition to casting the integer array to floating point, Pandas automati‐
cally converts the None to a NaN value. (Be aware that there is a proposal to add a
native integer NA to Pandas in the future; as of this writing, it has not been included.) <br>

While this type of magic may feel a bit hackish compared to the more unified
approach to NA values in domain-specific languages like R, the Pandas sentinel/cast‐
ing approach works quite well in practice and in my experience only rarely causes
issues.